In [82]:
%load_ext autoreload

%autoreload 2

import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
from math import sqrt

from ruspy.data.data_reading import data_reading
from ruspy.data.data_processing import data_processing

from ruspy.estimation.estimation import estimate
from ruspy.estimation.estimation_transitions import estimate_transitions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data_reading()

In [13]:
#Initilize a dictionary with values needed on the way.
np.random.seed(123)
beta = 0.9999
init_dict = {
    'groups': 'group_4',
    'beta': beta,
    'maint_func': 'linear',
    'binsize': 5000,
    'states': 90,
}

In [98]:
repl_data = data_processing(init_dict)
trans_results, cost_results = estimate(init_dict, repl_data)
print(trans_results, cost_results)

{'x': array([0.39189189, 0.59529357, 0.01281454]), 'fun': 3140.5705570938244, 'trans_count': [1682, 2555, 55]}       fun: 163.58583992537754
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([6.82121026e-05, 1.62003744e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 36
      nit: 10
   status: 0
  success: True
        x: array([10.0776928 ,  2.29412762])


In [99]:
p_ml = trans_results['x']

In [100]:
def get_x(p, l):
    return l - p[0] * sqrt( 1 + l ** 2 / 4) - 0.5 * l * p[1]

In [101]:
n = 4292
dim = 3
cov = np.zeros(shape=(dim, dim), dtype=float)
for i in range(dim):
    for j in range(dim):
        if i==j:
            cov[i, i] = n * p_ml[i] * (1 - p_ml[i])
        else:
            cov[i, j] = - n * p_ml[i] * p_ml[j]
cov

array([[ 1022.83783784, -1001.28378378,   -21.55405405],
       [-1001.28378378,  1034.0249301 ,   -32.74114632],
       [  -21.55405405,   -32.74114632,    54.29520037]])

In [19]:
# Specify simplex
l_simp = sqrt(4 / 3)

In [107]:
points = 100
draws_y = []
draws_x = []
draws_heat = []
draw = np.random.multivariate_normal(p_ml * n, cov, size=points)
for d in draw:
    probs = d / sum(d)
    draws_y += [probs[1]]
    draws_x += [get_x(probs, l_simp)]

In [ ]:
def creat_set(p_ml, cov_p, roh):
    

In [109]:
# Create triangulation.range(
x = np.array([0, 0.5 * l_simp, l_simp])
y = np.array([0, 1, 0])
triang = mtri.Triangulation(x, y)

# Set up the figure
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
ax.text(-0.05, -0.03, "Increase by 0")
ax.text(l_simp - 0.06, -0.03, "Increase by 2")
ax.text(0.5 * l_simp - 0.05, 1.02, "Increase by 1")
ax.plot([get_x(p_ml, l_simp)], [p_ml[1]], marker='o', markersize=1, color="red")
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax.scatter(draws_x, draws_y, s=1)
ax.triplot(triang)

FigureCanvasNbAgg()

In [105]:
# Create triangulation.range(
x = np.array([0, 0.5 * l_simp, l_simp])
y = np.array([0, 1, 0])
triang = mtri.Triangulation(x, y)

# Set up the figure
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
ax.text(-0.05, -0.03, "Increase by 0")
ax.text(l_simp - 0.06, -0.03, "Increase by 2")
ax.text(0.5 * l_simp - 0.05, 1.02, "Increase by 1")
ax.plot([get_x(p_ml, l_simp)], [p_ml[1]], marker='o', markersize=1, color="red")
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# Plot the triangulation.
#axs.tricontourf(triang, z)

# ax.set_title('Probability simplex')
# Plot heatmap
# ax2 = ax.twinx()
ax.hist2d(np.array(draws_x), np.array(draws_y), bins=2000, range=[[0, l_simp], [0, 1]])
ax.triplot(triang)

FigureCanvasNbAgg()

In [ ]:
cov